# PPP Loan Data > 150k

In [2]:
import pandas as pd
import pickle
import geocoder as geo
import folium as f
import requests
import re

In [29]:
df_loan = pd.read_csv('data/ppp_150.csv')

cols = ['LoanRange', 'BusinessName', 'Address', 'City', 'State', 'Zip', 'NAICSCode', 'BusinessType', 'JobsRetained']

In [30]:
df_loan_strip = df_loan[cols]

In [28]:
df_loan_strip.size

5950962

In [8]:
df_loan_strip.to_json('data/ppp_loan_strip.json')

In [1]:
df_loan_strip.to_csv('data/ppp_loan_strip.csv')

NameError: name 'df_loan_strip' is not defined

In [5]:
df_loan_strip = pd.read_csv("data/ppp_loan_strip.csv", index_col=0)

In [7]:
df_loan_strip.shape

(661218, 9)

In [17]:
df_loan_strip.isnull().sum()

LoanRange           0
BusinessName        8
Address            17
City               15
State               0
Zip                16
NAICSCode        6783
BusinessType     1429
JobsRetained    40506
dtype: int64

In [4]:
df_loan_strip.sort_values(by='LoanRange')

NameError: name 'df_loan_strip' is not defined

# Create Loan Type Category Column

In [46]:
df_loan_strip['LoanCat'] = df_loan_strip.apply(lambda row: re.search(r"[abcde]", row['LoanRange'])[0], axis=1)

<ipython-input-46-c732ee95b3bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loan_strip['LoanCat'] = df_loan_strip.apply(lambda row: re.search(r"[abcde]", row['LoanRange'])[0], axis=1)


In [47]:
del df_loan_strip['LoanRange']

In [105]:
na_addr = df_loan_strip[df_loan_strip['Address'].isna()]
na_city = df_loan_strip[df_loan_strip['City'].isna()]
na_zip = df_loan_strip[df_loan_strip['Zip'].isna()]
na_jobs = df_loan_strip[df_loan_strip['JobsRetained'].isna()]
zero_jobs = df_loan_strip[df_loan_strip['JobsRetained'] == 0]

In [106]:
na_city.shape, na_addr.shape, na_zip.shape, na_jobs.shape, zero_jobs.shape

((15, 9), (17, 9), (16, 9), (40506, 9), (48922, 9))

In [111]:
loan_type_list = ['a', 'b', 'c', 'd', 'e']

for l_type in loan_type_list:
    df_loan_strip[df_loan_strip['LoanCat'] == l_type].to_csv('data/loan_strip_cat' + l_type.capitalize() + '.csv')

### 13% of borrowers reported zero jobs retained

In [4]:
df_loan_strip[df_loan_strip['JobsRetained'].isna()].size/df_loan_strip.size

0.06125967532644302

In [5]:
df_loan_strip[df_loan_strip['JobsRetained'] == 0].size/df_loan_strip.size

0.07398770148423062

### Remove null and invalid states

In [15]:
df_addr = df_loan[['LoanRange', 'Address', 'City', 'State', 'Zip']].dropna()

In [16]:
df_addr[df_addr['State'] == 'XX']

,LoanRange,Address,City,State,Zip
661203,"d $350,000-1 million",1850 BOYER AVE E,SEATTLE,XX,98112.0
661204,"d $350,000-1 million",1661 MARTIN RANCH RD,SAN BERNARDINO,XX,92407.0
661205,"d $350,000-1 million",7684 Southrail Road,NORTH CHARLESTON,XX,29420.0
661206,"d $350,000-1 million",2410 Highway 15 South,SUMTER,XX,29150.0
661209,"e $150,000-350,000",501 State Road 13,SAINT JOHNS,XX,32259.0


In [17]:
df_addr = df_addr[df_addr['State'] != 'XX']

In [18]:
len(df_addr)

661195

# Geocoder lib

## Get address for A-Class loans

In [5]:
a_addr = pd.read_csv('data/loan_strip_catA.csv', index_col=0)

In [13]:
len(a_addr)

4840

def construct_full_addr(row):
    a_addr['full_addr'] = ((row['Address'].split(',')[0] + ', ' 
                            + row['City'] + ', ' + row['State']))

### Construct Full Addresses

`.split(',')` removes address appendices such as "Suite A", "Room 17", etc.

In [7]:
a_addr['full_addr'] = a_addr.apply(lambda row: (row['Address'].split(',')[0] + ', ' 
                            + row['City'] + ', ' + row['State']), axis=1)

### Get latitude and longitude for addresses of recipients of A-class loans ($5-10 million)

In [46]:
with requests.Session() as session:
    a_addr['latlng'] = a_addr.apply(lambda row: geo.osm(row['full_addr'], session=session).latlng, axis=1)

In [ ]:
pickle.dump(a_addr, open("a_adrr.pkd", "wb"))

In [24]:
a_addr = pickle.load(open("static/a_latlng.pkd", "rb"))

In [26]:
a_addr.shape, a_addr[a_addr['latlng'].isna()].shape

((4840, 11), (1770, 11))

## Get latlng for B-class loans

In [6]:
b_addr = pd.read_csv('data/loan_strip_catB.csv', index_col=0)

In [7]:
len(b_addr)

24838

### Construct Full Address Column

In [12]:
b_addr['full_addr'] = b_addr.apply(lambda row: (row['Address'].split(',')[0] + ', ' 
                            + row['City'] + ', ' + row['State']), axis=1)

### Request Latlng

In [23]:
with requests.Session() as session:
    b_addr['latlng'] = b_addr.apply(lambda row: geo.osm(row['full_addr'], session=session).latlng, axis=1)

In [25]:
pickle.dump(b_addr, open("static/b_addr.pkd", "wb"))

In [19]:
b_addr = pickle.load(open("static/b_addr.pkd", "rb"))

In [23]:
b_addr.shape, b_addr[b_addr['latlng'] ==].shape

((24838, 11), (0, 11))

### Pickling

### Useful function for checking actual size of an object

https://stackoverflow.com/questions/449560/how-do-i-determine-the-size-of-an-object-in-python

In [61]:
import sys
from types import ModuleType, FunctionType
from gc import get_referents

# Custom objects know their class.
# Function objects seem to know way too much, including modules.
# Exclude modules as well.
BLACKLIST = type, ModuleType, FunctionType


def getsize(obj):
    """sum size of object & members."""
    if isinstance(obj, BLACKLIST):
        raise TypeError('getsize() does not take argument of type: '+ str(type(obj)))
    seen_ids = set()
    size = 0
    objects = [obj]
    while objects:
        need_referents = []
        for obj in objects:
            if not isinstance(obj, BLACKLIST) and id(obj) not in seen_ids:
                seen_ids.add(id(obj))
                size += sys.getsizeof(obj)
                need_referents.append(obj)
        objects = get_referents(*need_referents)
    return size

# Map

In [70]:
# create map
m = f.Map(location=[44.58, -103.46], zoom_start=3)

# global tooltip
tooltip = 'Click for more info'

# generate map
m.save('map.html')

In [71]:
def add_aClass_loans(latlng):
    f.CircleMarker(
        location=latlng,
        radius=2,  # use for loan size? scale to size of loan? normalize these?
        popup="$5-10 Million",  # loan value
        color='blue',  # color according to some histogram?
        fill=False,
        fill_color='blue'
    ).add_to(m)

In [72]:
def add_loans_to_map():
    for i in a_latlng.itertuples():
        if i.latlng != None:
            add_aClass_loans(i.latlng)
#             print(i.latlng)

In [35]:
a_latlng[a_latlng['latlng'].isna()]

,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,JobsRetained,LoanCat,full_addr,latlng
5,"NANA WORLEY, LLC","3700 Centerpoint Drive, 7th floor",ANCHORAGE,AK,99503.0,541330.0,Limited Liability Company(LLC),231.0,a,"3700 Centerpoint Drive, ANCHORAGE, AK",None
11,"YULISTA TACTICAL, LLC",5015 BUSINESS PARK BLVD Suite 3000,ANCHORAGE,AK,99503.0,541614.0,Limited Liability Company(LLC),220.0,a,"5015 BUSINESS PARK BLVD Suite 3000, ANCHORAGE, AK",None
13,"ALLIANCE HR, INC",307 CLINTON AVE suite 200,HUNTSVILLE,AL,35801.0,561311.0,Corporation,443.0,a,"307 CLINTON AVE suite 200, HUNTSVILLE, AL",None
19,ATS OPERATING LLC,1900 CRESTWOOD BLVD SUITE 302,IRONDALE,AL,35210.0,453310.0,Limited Liability Company(LLC),250.0,a,"1900 CRESTWOOD BLVD SUITE 302, IRONDALE, AL",None
22,BEVILACQUA RESEARCH CORPORATION,4901 CORPORATE DR Ste B,HUNTSVILLE,AL,35805.0,541330.0,Corporation,343.0,a,"4901 CORPORATE DR Ste B, HUNTSVILLE, AL",None
...,...,...,...,...,...,...,...,...,...,...,...
4826,"LEXINGTON COAL COMPANY, INC",1051 Main St,MILTON,WV,25541.0,213113.0,Limited Liability Company(LLC),139.0,a,"1051 Main St, MILTON, WV",None
4828,NOBORNE PRESCHOOL AND DAYCARE CENTER,200 KING ST,MARTINSBURG,WV,25401.0,624410.0,Non-Profit Organization,17.0,a,"200 KING ST, MARTINSBURG, WV",None
4831,TRITON CONSTRUCTION INC,1944 WINFIELD RD,SAINT ALBANS,WV,25177.0,237310.0,Subchapter S Corporation,264.0,a,"1944 WINFIELD RD, SAINT ALBANS, WV",None
4832,"CYCLONE DRILLING, INC.",5800 Mohan Road PO Box 908,GILLETTE,WY,82717.0,213112.0,Corporation,364.0,a,"5800 Mohan Road PO Box 908, GILLETTE, WY",None


In [73]:
add_loans_to_map()

In [74]:
m.save('map.html')

# Separate by loan range

In [75]:
dollars = re.compile(r"(?P<amt>\d+)|(?P<mag>million|thousand)")

In [84]:
re.search(dollars, loan)

<re.Match object; span=(3, 4), match='5'>

In [94]:
lr = loan_range = re.findall(dollars, loan)
lr = str(1.*(int(lr[1][0]) + int(lr[0][0]))/2) + ' ' + lr[2][1]

In [104]:
a = re.search(lr, r"\f")

In [106]:
lr

'7.5 million'

In [60]:
lat, long = geo[0]['geometry']['location']['lat'], geo[0]['geometry']['location']['lng']
m = f.Map([lat, long], zoom_start=12)

In [109]:
f.CircleMarker(
    location=[lat, long],
    radius=7.5,  # use for loan size? scale to size of loan? normalize these?
    popup=str(lr),  # loan value
    color='blue',  # color according to some histogram?
    fill=True,
    fill_color='blue'
).add_to(m)

In [110]:
m.save('ppp_test_map.html')